# Demonstration: Dislocation Creation

**Lucas M. Hale**, [lucas.hale@nist.gov](mailto:lucas.hale@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Chandler A. Becker**, [chandler.becker@nist.gov](mailto:chandler.becker@nist.gov?Subject=ipr-demo), *Materials Science and Engineering Division, NIST*.

**Zachary T. Trautt**, [zachary.trautt@nist.gov](mailto:zachary.trautt@nist.gov?Subject=ipr-demo), *Materials Measurement Science Division, NIST*.

Version: 2015-08-28

[Disclaimers](http://www.nist.gov/public_affairs/disclaimer.cfm)

- - -

## Software

This notebook was tested with:

- LAMMPS (Version 2015-03-27)

- Python (Version 2.7.6)

- IPython (Version 2.0.0)

- - -

## Introduction

This Notebook uses the Stroh method for solving the Eshelby anisotropic elasticity solution for a straight line displacement (i.e. full dislocation). 

Related calculations:

- tool-Dislocation-Nye-Tensor: calculates the Nye Tensor for all atoms in a system.

- tool-Dislocation-Differential-Displacement: creates a differential displacement plot.


- - -

##1. Initial Setup

1. __Necessary Python Libraries__: This is a list of the Python libraries used to run this code. 

2. __Necessary Parameters__: These are the parameters that need to be specified to run the code.

3. __Calculation Parameters__: These are user-specified parameters that control how the calculation proceeds, such as number of data points.

###1.1 Necessary Python Libraries

This is a list of the Python libraries used to run this code. 

- Numpy can be found [here](http://www.numpy.org/).

- iprp is contained in the IPR code download.

In [1]:
#Standard Python Libraries
import subprocess
import os
from copy import deepcopy
import sys
from math import ceil

#Additional Python Libraries
import numpy as np

#Custom package
import atomman as am
from atomman.tools import mag

###1.2 Necessary Parameters

These are the parameters that need to be specified to run the code.

- lammps_exe is the directory location for the LAMMPS executable to use.

- working_dir is the working directory where you want this Notebook to run.

- pot_file is the name (and location) of the interatomic potential data model file associated with the potential you want to use. A default potential (Al-Si-Mg-Cu-Fe--JelinekB--2012) is embedded in Section 2.1 of this Notebook, and the associated interatomic potential data model file is embedded in Section 2.2. Other potentials and interatomic potential data models can be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/).

- pot_dir is the directory location where any potential artifacts (i.e. eam setfl, meam library files) are located.

- proto_file is the name (and location) of a crystal prototype data model file. These files contain sets of parameters associated with generating different crystal prototypes. One example (fcc.json) is embedded in Section 2.3. Other crystal prototype data models can be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/). 

- elements is a list of the elements to be associated with the unique lattice sites of the selected crystal prototype. Elemental prototypes (eg. fcc) will need only one element, while compounds (eg. CsCl) require multiple elements.

- alat is an array of the three lattice parameters.  

- cij is the (6x6) matrix representation of the elastic constants associated with \[100] \[010] \[001] crystallographic axes orientations.

- disl_file is the name (and location) of a dislocation monopole library data model file.  These files contain parameters associated with generating different dislocation monopole systems. One example (fcc-disl.json) is embedded in Section 2.4. Other dislocation monopole library data models can be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/). 

In [2]:
#Specify LAMMPS run command
lammps_exe = 'C:\\users\\lmh1\\Documents\\lmp_serial.exe'

#Specify working directory
working_dir = 'tool-Dislocation'
try:
    os.chdir(working_dir)
except:
    os.mkdir(working_dir)
    os.chdir(working_dir)
working_dir = os.getcwd()

#Specify interatomic potential data model file
pot_file = 'Al-Si-Mg-Cu-Fe--JelinekB--2012.json'

#Specify path to potential artifacts, (eam setfl file, meam library files, etc.)
pot_dir = working_dir

#Specify crystal prototype data model file
proto_file = 'fcc.json'

#List the element(s) to use
symbols = ['AlS']

#Give the lattice parameters
a = b = c = 4.0500
alpha = beta = gamma = 90.0

#Specify the elastic constant matrix
cij = np.array(   [[ 110.53,   60.90,   60.90,    0.00,    0.00,    0.00],
                   [  60.90,  110.53,   60.90,    0.00,    0.00,    0.00],
                   [  60.90,   60.90,  110.53,    0.00,    0.00,    0.00],
                   [   0.00,    0.00,    0.00,   28.39,    0.00,    0.00],
                   [   0.00,    0.00,    0.00,    0.00,   28.39,    0.00],
                   [   0.00,    0.00,    0.00,    0.00,    0.00,   28.39]]  )

#Specify point defect library data model file
disl_file = 'fcc-disl.json'

#Specify which dislocation type to create and analyze
#a/2<110> screw:
disl_type = '(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(0)(-1)]'
#a/2<110> edge:
#disl_type = '(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(-2)(1)]'

###1.3 Calculation Parameters

These are user-specified parameters that control how the calculation proceeds, such as number of data points.

- system_size is a set of system dimension multipliers to use in the creation of the system.  Each dimension, i, will be at least system_size[i] \* alat[i]. 

- temperature sets the temperature at which to perform the annealing.  If 0, then only a minimization relaxation is performed.  For other temperatures, an NVT run for 10 ps at that temperature is performed before minimization.

- boundary_width defines how thick to make the boundary region in the xy plane. 

- boundary_shape specifies how to define the boundary region relative to the active MD region.  If 'circle', then the MD region will be a cylinder where the circular cross section is at least boundary_width from the x and y box edges. If 'rectangle', then the boundary region is defined as being b_width thick from the x and y box edges all the way around. 

__Note 1:__ Using a fixed boundary region means that there are incompatibility forces across the MD-fixed region interface.  These will result in a long-range feedback force on the dislocation keeping it near the center of the MD region.  If strong enough, it may influence the core structure. This effect can be reduced by increasing the x and y dimensions in the system size.

__Note 2:__ There is no guarantee that the obtained relaxed core structure is the true lowest energy stable configuration. Creating large systems and performing a temperature relaxation will help. However, note that the temperature relaxation is performed using the 0K lattice and elastic constants which may not be entirely realistic.  

In [3]:
#Give system size multipliers.  Each dimension will be at least that many times the a lattice parameter.
system_size = [40, 40, 1]

#Give the relaxation temperature.  
#temperature = 0 will do a minimization 
#temperature > 0 will do nvt relaxation followed by minimization (takes longer)
temperature = 0

#Set the minimum width of the fixed boundary region.
boundary_width = 3 * a

#Specify the shape to use in defining the boundary region.
boundary_shape = 'circle'

##2. Additional Data Files

To make this notebook self-contained, we have embedded a few files.

1. __MEAM Potential.__ Generates library and parameter files for one MEAM potential.

2. __Potential Data Model.__ Collects all input parameters assoiciated with running the MEAM potential in LAMMPS.

3. __Prototype Data Model.__ Generates an instance of the crystal prototype data model used to generate various structures.

4. __Dislocation Monopole Library Data Model.__ Collects input parameters associated with generating a number of different dislocation monopoles.

###2.1 MEAM Potential

This generates the library and parameter files associated with the following interatomic potential taken from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/):

__Al-Si-Mg-Cu-Fe--JelinekB--2012__

__Aluminum, Silicon, Magnesium, Copper, and Iron (Al, Si, Mg, Cu, and Fe) Alloys__

*B. Jelinek, S. Groh, M. Horstemeyer, J. Houze, S.G. Kim, G.J. Wagner, A. Moitra, and M.I. Baskes, "Modified embedded atom method potential for Al, Si, Mg, Cu, and Fe alloys," Phys. Rev. B 85, 245102 (2012).*

__Notes__: This file was provided by Bohumir Jelinek (Mississippi State University) and posted on 3 July 2012. He noted, "This is a MEAM potential for Al, Si, Mg, Cu, Fe alloys. It works with LAMMPS, version 19 Jul 2011 or later, when compiled with MEAM support."

__Notice__: Users should consider all available potentials and select one which is appropriate for their use case. Use of this potential within this demonstration should not be construed as an endorsement or a recommendation. 

In [4]:
#Create MEAM library and parameter files for the Al-Si-Mg-Cu-Fe--JelinekB--2012 potential
   
#Create MEAM library file "Jelinek_2012_meamf"
f = open('Jelinek_2012_meamf', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meam.alsimgcufe" file providing combinational parameters
#  elt        lat     z       ielement     atwt
#  alpha      b0      b1      b2           b3    alat    esub    asub
#  t0         t1              t2           t3            rozero  ibar

'AlS'        'fcc'   12.     13           26.9815
4.64        2.04    3.00    6.0          1.50  4.05    3.353    1.07
1.0         +4.50           -2.30        8.01          1.0    -5
'SiS'        'dia'   4.      14           28.086
4.87        4.4     5.5     5.5          5.5   5.431   4.63    1.
1.0         2.05            4.47         -1.80         2.2    -5
'MgS'       'hcp'   12.      12           24.305
5.52        4.0    3.0     0.2          1.2  3.194 1.51     0.80
1.0         10.04           9.49         -4.3          0.63   -5
'CuS'        'fcc'   12.     29           63.54
5.11        3.634   2.20    6            2.20  3.62    3.54    1.07
1.0         4.91            2.49         2.95          1.10   -5
'FeS'        'bcc'   8       26           55.847
5.0270      3.500   2       1.00         1     2.851   4.28    0.5550
1          -1.6             12.5          -1.40        1.0    -5""")
f.close()

#Create MEAM parameter file "Jelinek_2012_meam.alsimgcufe"
f = open('Jelinek_2012_meam.alsimgcufe', 'w')
f.write("""# MEAM Al, Si, Mg, Cu, Fe alloy potential, Phys. Rev. B 85, 245102 (2012)
# http://dx.doi.org/10.1103/PhysRevB.85.245102
# to be used with "meamf" file providing single element parameters

  Cmin(1,1,1) = 0.8
  repuls(1,1) = 0.1
  Cmin(3,3,3) = 0.8
  Cmin(4,4,4) = 0.8
  Cmin(5,5,5) = 0.68
  repuls(5,5) = 0.3
  Cmax(5,5,5) = 1.9

  nn2(1,1)=1
  nn2(1,2)=1
  nn2(1,3)=1
  nn2(1,4)=1
  nn2(1,5)=1
  nn2(2,2)=1
  nn2(2,3)=1
  nn2(2,4)=1
  nn2(2,5)=1
  nn2(3,3)=1
  nn2(3,4)=1
  nn2(3,5)=1
  nn2(4,4)=1
  nn2(4,5)=1
  nn2(5,5)=1

  lattce(1,2)='b1'
  delta(1,2)=+0.28
  alpha(1,2)=4.56
  re(1,2)=2.62
  Cmin(1,1,2) = 0.50
  Cmin(2,2,1) = 2.00
  Cmin(1,2,1) = 2.00
  Cmin(1,2,2) = 2.00

  lattce(1,3)='b1'
  delta(1,3)=+0.23
  alpha(1,3)=4.52
  re(1,3)=2.87
  Cmin(1,1,3) = 2.00
  Cmin(3,3,1) = 0.00
  Cmin(1,3,1) = 2.00
  Cmin(1,3,3) = 0.00

  lattce(1,4)='b1'
  delta(1,4)=+0.19
  alpha(1,4)=4.65
  re(1,4)=2.53
  Cmin(1,1,4) = 0.00
  Cmin(4,4,1) = 2.00
  Cmin(1,4,1) = 2.00
  Cmin(1,4,4) = 2.00

  lattce(1,5)='b1'
  delta(1,5)=+0.26
  alpha(1,5)=4.64
  re(1,5)=2.45
  Cmin(1,1,5) = 0.90
  Cmin(5,5,1) = 0.10
  Cmin(1,5,1) = 2.00
  Cmin(1,5,5) = 2.00

  lattce(2,3)='b1'
  delta(2,3)=+0.2
  alpha(2,3)=4.73
  re(2,3)=2.75
  Cmin(2,2,3) = 1.00
  Cmin(3,3,2) = 1.00
  Cmin(2,3,2) = 2.00
  Cmin(2,3,3) = 2.00

  lattce(2,4)='b1'
  delta(2,4)=+0.14
  alpha(2,4)=4.74
  re(2,4)=2.46
  Cmin(2,2,4) = 0.00
  Cmin(4,4,2) = 0.00
  Cmin(2,4,2) = 2.00
  Cmin(2,4,4) = 2.00

  lattce(2,5)='b1'
  delta(2,5)=-0.07
  alpha(2,5)=5.17
  re(2,5)=2.39
  Cmin(2,2,5) = 1.00
  Cmin(5,5,2) = 1.00
  Cmin(2,5,2) = 2.00
  Cmin(2,5,5) = 0.00
  attrac(2,5) = 0.1
  repuls(2,5) = 0.1

  lattce(3,4)='b1'
  delta(3,4)=+0.23
  alpha(3,4)=4.70
  re(3,4)=2.63
  Cmin(3,3,4) = 2.00
  Cmin(4,4,3) = 0.00
  Cmin(3,4,3) = 2.00
  Cmin(3,4,4) = 2.00

  lattce(3,5)='b1'
  delta(3,5)=+0.6
  alpha(3,5)=4.96
  re(3,5)=2.61
  Cmin(3,3,5) = 0.65
  Cmin(5,5,3) = 0.00
  Cmin(3,5,3) = 2.00
  Cmin(3,5,5) = 2.00

  lattce(4,5)='b1'
  delta(4,5)=+0.63
  alpha(4,5)=5.21
  re(4,5)=2.42
  Cmin(5,5,4)=0.00

  attrac(5,2) = 0.1
  repuls(5,2) = 0.1

  rc = 5.0
  ialloy=1
  augt1=0
  delr=0.25658351
  emb_lin_neg=1
  bkgd_dyn=1""")
f.close()

###2.2 Potential Data Model

This generates the interatomic potential data model file associated with the Al-Si-Mg-Cu-Fe--JelinekB--2012 potential.  Additional interatomic potential data model files can also be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/).

In [5]:
#Creates a interatomic potential data model for using the MEAM potential in LAMMPS
f = open('Al-Si-Mg-Cu-Fe--JelinekB--2012.json', 'w')
f.write("""{
    "interatomicPotentialImplementationLAMMPS": {
        "potentialID": {
            "descriptionIdentifier": "Al-Si-Mg-Cu-Fe--JelinekB--2012"
        },
        "units": "metal",
        "atom_style": "atomic",
        "atom": [
            {
                "element": "Al",
                "symbol": "AlS"
            },
            {
                "element": "Si",
                "symbol": "SiS"
            },
            {
                "element": "Mg",
                "symbol": "MgS"
            },
            {
                "element": "Cu",
                "symbol": "CuS"
            },
            {
                "element": "Fe",
                "symbol": "FeS"
            }
        ],
        "pair_style": {
            "type": "meam"
        },
        "pair_coeff": {
            "term": [
                {
                    "file": "Jelinek_2012_meamf"
                },
                {
                    "symbolsList": "True"
                },
                {
                    "file": "Jelinek_2012_meam.alsimgcufe"
                },
                {
                    "symbols": "True"
                }
            ]
        }
    }
}""")
f.close()    

###2.3 Prototype Data Model

This generates the crystal prototype data model file associated with the fcc crystal prototype.  Additional crystal prototype data model files can also be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/).

In [6]:
#Creates an instance of a crystal prototype data model for generating crystal structures.
f = open('fcc.json', 'w')
f.write("""{
    "crystalPrototype": {
        "crystalProtoypeID": { 
            "common": "face-centered cubic", 
            "tag": "fcc",
            "prototype": "Cu",
            "Strukturbericht": "A1"
        },            
        "crystalProtoypeInfo": { 
            "PearsonSymbol": "cF4",
            "spaceGroupTag": "F m -3 m",
            "SpaceGroupNumber": 225
        },
        "lattice": {
            "cubic": {
                "atomPositions": {
                    "site": {
                        "component": 1,
                        "atomCoordinates": [
                            {
                                "value": [0.0000000000000, 0.0000000000000, 0.0000000000000],
                                "unit": "scaled"
                            },                      
                            {
                                "value": [0.5000000000000, 0.5000000000000, 0.0000000000000],
                                "unit": "scaled"
                            },
                            {
                                "value": [0.5000000000000, 0.0000000000000, 0.5000000000000],
                                "unit": "scaled"
                            },
                            {
                                "value": [0.0000000000000, 0.5000000000000, 0.5000000000000],
                                "unit": "scaled"
                            }
                        ]
                    }
                }
            }
        }
    }
}""")
f.close()

###2.4 Dislocation Monopole Library Data Model

This generates the dislocation monopole library data model file associated with dislocations in the fcc crystal. Additional dislocation monopole library data model files can also be downloaded from the [NIST Interatomic Potential Repository](http://www.ctcms.nist.gov/potentials/).

In [7]:
#Creates an instance of a point defect library data model for generating point defects.
f = open('fcc-disl.json', 'w')
f.write("""{
    "dislocationMonopoleList": {
        "crystalPrototype": "fcc",
        "nyeTensorParameters": {
            "nearestNeighborCutoff": {
                "value": 0.8535533905933,
                "unit": "scaled"
            },
            "maximumBondAngle": {
                "value": 27,
                "unit": "degree"
            },
            "referenceNeighborVectors": {
                "site": {
                    "neighborVectors": [
                        {
                            "value": [ 0.0, 0.5, 0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.0,-0.5, 0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.0,-0.5,-0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.0, 0.5,-0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.5, 0.0, 0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [-0.5, 0.0, 0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [-0.5, 0.0,-0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.5, 0.0,-0.5],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.5, 0.5, 0.0],
                            "unit": "scaled"
                        },
                        {
                            "value": [-0.5, 0.5, 0.0],
                            "unit": "scaled"
                        },
                        {
                            "value": [-0.5,-0.5, 0.0],
                            "unit": "scaled"
                        },
                        {
                            "value": [ 0.5,-0.5, 0.0],
                            "unit": "scaled"
                        }
                    ]
                }
            }
        },
        "dislocationMonopole": [
            {
                "dislocationMonopoleInformation": {
                    "tag": "(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(0)(-1)]",
                    "burgersVector": "a/2[ 1, 0,-1]",
                    "slipPlane": [ 1, 1, 1],
                    "lineDirection": [ 1, 0,-1],
                    "character": "screw"
                },
                "simulationParameters": {
                    "burgers": [ 0.5, 0.0,-0.5],
                    "crystallographicAxes": {
                        "x-axis": [-1, 2,-1],
                        "y-axis": [ 1, 1, 1],
                        "z-axis": [ 1, 0,-1]
                    },
                    "shift": [ 0.00000000000000, 0.50000000000000, 0.00000000000000],
                    "width": [-0.35708892449921, 0.35708892449921]
                }
            },
            {
                "dislocationMonopoleInformation": {
                    "tag": "(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(-2)(1)]",
                    "burgersVector": "a/2[ 1, 0,-1]",
                    "slipPlane": [ 1, 1, 1],
                    "lineDirection": [ 1,-2, 1],
                    "character": "edge"
                },
                "simulationParameters": {
                    "burgers": [ 0.5, 0.0,-0.5],
                    "crystallographicAxes": {
                        "x-axis": [ 1, 0,-1],
                        "y-axis": [ 1, 1, 1],
                        "z-axis": [ 1,-2, 1]
                    },
                    "shift": [ 0.12500000000000, 0.50000000000000, 0.00000000000000],
                    "width": [-0.61849616005276, 0.61849616005276]
                }
            }
        ]   
    }
}""")
f.close()

##3. LAMMPS Script Generation Function(s)

This code generates the underlying LAMMPS script(s) for performing the necessary simulations.  

- disl_relax_script() is used to relax the dislocation monopole system.  In doing so, the atoms of the MD and fixed regions are defined according to atom types, where the first half of types are in the active MD group 'move', and the second half og types are in the fixed group 'hold'.  If temperature is not 0, then an NVT run for 10 ps at that temperature is performed prior to minimization.   

In [8]:
def disl_relax_script(system_info, pair_info, moving_atypes, temp = 0):
    #Perform energy minimization (and possibly NVT) on dislocation monopole system in file read_data
    
    group_move = ''
    for atype in moving_atypes:
        group_move += ' '+str(atype)

    newline = '\n'
    script = newline.join([system_info,
                           '',
                           pair_info,
                           '',
                           'group move type' + group_move,
                           'group hold subtract all move',
                           '',
                           'compute peatom all pe/atom',
                           '',
                           'dump first all custom 100000 atom.* id type x y z c_peatom',
                           'dump_modify first format "%d %d %.13e %.13e %.13e %.13e"',
                           'thermo_style custom step pe',
                           ''])
    if temp == 0:
        script = newline.join([script, 
                               'fix nomove hold setforce 0.0 0.0 0.0',
                               'minimize 0 1e-5 10000 100000'])
    else:
        script = newline.join([script,
                               'velocity move create %f 9467 mom yes rot yes dist gaussian'%(2 * temp),
                               'fix nomove hold setforce 0.0 0.0 0.0',
                               'timestep 0.001',
                               'thermo 10000',
                               'fix 1 all nvt temp %f %f 0.1'%(temp, temp),
                               '',
                               'run 10000',
                               'minimize 0 1e-5 10000 100000'])  
    return script

##4. Python Calculation Function(s)

These functions are associated with setting up the atomic systems, running LAMMPS, extracting the simulation data, and refining the results.

1. __System Modification.__ Adjusts the system so that disl_relax script will work properly.

2. __Stroh Method.__ Functions associated with obtaining the Eshelby elastic solution using the Stroh method.

###4.1 System Modification

- boundaryfix() modifies the box dimensions and the atom types in the system.  Inserting the defect can cause atoms to be displaced outside the original box, so the xy bounds are shifted to accomodate this.  Additionally, the atom types are changed such that the disl_relax script can identify the boundary region atoms.  

In [9]:
def boundaryfix(system, b_width, shape):
    #Modify the box size and atom types to support the boundary conditions for disl_relax_script
    
    natypes = system.natypes()
    
    if shape == 'circle':
        #find x or y bound closest to 0
        smallest_xy = min([abs(system.box('xlo')), 
                           abs(system.box('xhi')),
                           abs(system.box('ylo')),
                           abs(system.box('yhi'))])
        
        radius = smallest_xy - b_width
        for i in xrange(system.natoms()):
            if mag(system.atoms(i,'pos')[:2]) > radius:
                system.atoms(i, 'atype', system.atoms(i, 'atype') + natypes)
    
    elif shape == 'rect':
        for i in xrange(system.natoms()):
            if (system.atoms(i, 'pos', 0) < system.box('xlo') + b_width or 
                system.atoms(i, 'pos', 0) > system.box('xhi') - b_width or 
                system.atoms(i, 'pos', 1) < system.box('ylo') + b_width or 
                system.atoms(i, 'pos', 1) > system.box('yhi') - b_width):
           
                system.atoms(i, 'atype', system.atoms(i, 'atype') + natypes)
           
    else:
        raise ValueError("Unknown shape type! Enter 'circle' or 'rect'")

###4.2 Stroh Method

This section contains the functions associated with the Stroh method for solving the Eshelby anisotropic elasticity solution.

- ax_check() checks that a crystallographic axes set is orthogonal and returns both the associated normalization vectors and the magnitudes associated with those directions.  Note that the normalized vector array is the transformation matrix, T, relative to a [1,0,0],[0,1,0],[0,0,1] orientation.

- c_transform() transforms the (3x3x3x3) elastic constant matrix, C, using the transformation matrix, T

- c_mn_to_c_ijkl() converts the (6x6) representation of the elastic constant matrix to the (3x3x3x3) representation. 

- c_ijkl_to_c_mn() converts the (3x3x3x3) representation of the elastic constant matrix to the (6x6) representation. 

- stroh_setup() takes the (3x3x3x3) elastic constant matrix transformed wrt Cartesian directions and finds the six complex roots of the Eshelby solution. For the solution, the defect is taken to be parallel to the z-axes. If all validation checks pass, it will return a dictionary containing terms that can then be used in the following stroh calcuation functions.

- stroh_preln() calculates and returns the pre-logarithmic factor assoicated with the dislocation's elastic energy. Requires the Burgers vector wrt Cartesian directions, b, and the Stroh solution dictionary, sd.  

- stroh_disp_point() computes the Stroh solution for the displacement at a point in the system. As the defect is parallel to the z-axis, the point is identified only with x and y coordinates, where the defect is placed at xy = (0,0). Requires the Burgers vector wrt Cartesian directions, b, and the Stroh solution dictionary, sd.

- stroh_stress_point() computes the Stroh solution for the stress state at a point in the system. As the defect is parallel to the z-axis, the point is identified only with x and y coordinates, where the defect is placed at xy = (0,0). Requires the Burgers vector wrt Cartesian directions, b, the elastic constant matrix transformed wrt Cartesian directions, C, and the Stroh solution dictionary, sd.

In [10]:
def axes_check(axes, tol=1e-8):
    #Checks axis relationship and returns transformation matrix, T, and axis magnitudes, mag
    
    mag = np.apply_along_axis(np.linalg.norm, 1, axes)
    uaxes = axes / mag[:,None]
    if (np.isclose(np.dot(uaxes[0], uaxes[1]), 0., atol=tol) == False or 
        np.isclose(np.dot(uaxes[0], uaxes[2]), 0., atol=tol) == False or 
        np.isclose(np.dot(uaxes[1], uaxes[2]), 0., atol=tol) == False):
        raise ValueError('dots are not 0!')
    if np.allclose(np.cross(uaxes[0], uaxes[1]) - uaxes[2], np.zeros(3), atol=tol) == False:
        raise ValueError('cross does not check!')
    return uaxes, mag

class Stroh():
    #Object for calculating the Stroh anisotropic elasticity solution for a perfectly straight dislocation
    
    def __init__(self, C, b, tol=1e-8): 
        assert isinstance(C, am.tools.ElasticConstants)
        self.C = C
        
        self.burgers = np.array(b, dtype=float)
        assert self.burgers.shape = (3,)
        
        assert isinstance(tol, float)
        self.tol = tol
        
        self.setup()

    def setup():
        #Does the initial Stroh calculations and returns a dictionary containing system wide parameters
        C = self.C.Cijkl()
        tol = self.tol
        
        #Matrixes of Cijkl constants used to construct N
        m = np.array([1.0, 0.0, 0.0])
        n = np.array([0.0, 1.0, 0.0])
        mm = np.einsum('i,ijkl,l', m, C, m)
        mn = np.einsum('i,ijkl,l', m, C, n)
        nm = np.einsum('i,ijkl,l', n, C, m)
        nn = np.einsum('i,ijkl,l', n, C, n)
        
        #The four 3x3 matrixes that represent the quadrants of N
        NB = -np.linalg.inv(nn)
        NA = NB.dot(nm)
        NC = mn.dot(NA) + mm
        ND = mn.dot(NB)
        
        #N is the 6x6 array, where the eigenvalues are the roots p
        #and the eigenvectors give A and L
        N =  np.array(np.vstack((np.hstack((NA, NB)), np.hstack((NC, ND)))))
        
        #Calculate the eigenvectors and eigenvalues
        eig = np.linalg.eig(N)
        self.p = eig[0]
        eigvec = np.transpose(eig[1])
        
        #round out near zero terms
        self.p.real[abs(self.p.real) < tol] = 0.0
        self.p.imag[abs(self.p.imag) < tol] = 0.0
        eigvec.real[abs(eigvec.real) < tol] = 0.0
        eigvec.imag[abs(eigvec.imag) < tol] = 0.0

        #separate the eigenvectors into A and L
        self.A = np.array([eigvec[0,:3], eigvec[1,:3], eigvec[2,:3], eigvec[3,:3], eigvec[4,:3], eigvec[5,:3]])
        self.L = np.array([eigvec[0,3:], eigvec[1,3:], eigvec[2,3:], eigvec[3,3:], eigvec[4,3:], eigvec[5,3:]])
        
        #calculate k
        self.k = 1. / (2. * np.einsum('si,si->s', self.A, self.L))
        
        #Checks and elastic coefficient calculations
        Check1 = np.einsum('s,si,sj->ij', self.k, self.A, self.L)
        Check2 = np.einsum('s,si,sj->ij', self.k, self.A, self.A)
        Check3 = np.einsum('s,si,sj->ij', self.k, self.L, self.L)
        Check4 = np.einsum('s,t,si,ti->st', self.k**.5, self.k**.5, self.A, self.L) + np.einsum('s,t,ti,si->st', self.k**.5, self.k**.5, self.A, self.L)
        
        #Round away near zero terms
        Check1.real[abs(Check1.real) < tol] = 0.0
        Check1 = np.real_if_close(Check1, tol = tol)
        Check2.real[abs(Check2.real) < tol] = 0.0
        Check2 = np.real_if_close(Check2, tol = tol)
        Check3.real[abs(Check3.real) < tol] = 0.0
        Check3 = np.real_if_close(Check3, tol = tol)    
        Check4.real[abs(Check4.real) < tol] = 0.0
        Check4.imag[abs(Check4.imag) < tol] = 0.0
        Check4 = np.real_if_close(Check4, tol=tol)
        
        #Verify checks passed
        assert np.allclose(Check1, np.identity(3),   atol=tol), 'Stroh checks failed!'
        assert np.allclose(Check2, np.zeros((3, 3)), atol=tol), 'Stroh checks failed!'
        assert np.allclose(Check3, np.zeros((3, 3)), atol=tol), 'Stroh checks failed!'
        assert np.allclose(Check4, np.identity(6),   atol=tol), 'Stroh checks failed!'   
        
    def preln(self):
        #Calculate the pre-ln energy factor in eV
        
        ii = np.array([1.j])
        updn = np.array([1, -1, 1, -1, 1, -1])
        
        coeff = ii * np.einsum('s,s,si,sj->ij', updn, self.k, self.L, self.L)
        coeff.real[abs(coeff.real) < self.tol] = 0.0
        coeff = np.real_if_close(coeff, tol=self.tol)
        
        return self.burgers.dot(coeff.dot(self.burgers)) * 4.96683295e-4

    def displacement_point(self, x, y):
        #Calculate the Stroh displacement associated with x,y coordinates for one point
        
        ii = np.array([1.j])
        updn = np.array([1, -1, 1, -1, 1, -1])

        Nu = x + self.p * y
        disp = 1 / (2 * np.pi * ii) * np.einsum('a,a,ai,a,a->i', updn, self.k, self.A, self.L.dot(self.burgers), np.log(Nu))
        disp.real[abs(disp.real) < self.tol] = 0.0
        disp = np.real_if_close(disp, tol=self.tol)
        return disp

    def stress_point(self, x, y):
        #Calculate the Stroh stress solution associated with x,y coordinates for one point
        C = self.C.Cijkl()
        
        ii = np.array([1.j])
        updn = np.array([1, -1, 1, -1, 1, -1])
        
        Nu = x + sd['p'] * y
        mpn = np.array([1.0, 0.0, 0.0]) + np.einsum('a,l->al', sd['p'], np.array([0.0, 1.0, 0.0]))
        stress = 1 / (2 * np.pi * ii) * np.einsum('a,a,ijkl,al,ak,a,a->ij', updn, sd['k'], C, mpn, sd['A'], sd['L'].dot(self.burgers), 1/Nu)
        stress.real[abs(stress.real) < self.tol] = 0.0
        stress = np.real_if_close(stress, tol=self.tol)
        return stress

##5. Run Calculation and Display Results

This is the code that runs the calculation function(s) for the supplied inputs and displays the resulting data.

1. __Create Base System.__ This section creates a properly oriented system for insertion of the defect.

2. __Stroh Calculations.__ This section uses the Stroh method to find the elasticity solution.

3. __Dislocation System Construction.__ This section creates a system containing a dislocation monopole by shifting atoms according to the Stroh displacement.

###5.1 Create Base System

This section creates a properly oriented system for insertion of the defect.

In [11]:
#Read the data model files
potential = am.lammps.Potential(pot_file, pot_dir)
prototype = am.tools.Prototype(proto_file)
dislocations = am.tools.DislocationMonopole(disl_file)

#Define run parameters
ucell = prototype.ucell(a=a, b=b, c=c, alpha=alpha, beta=beta, gamma=gamma)
axes = dislocations.get(disl_type, 'axes')
shift = dislocations.get(disl_type, 'shift')

T, ax_mag = axes_check(axes)

#Set system size parameters
s = np.zeros(3, dtype=np.int)
for i in xrange(3):
    scale = ceil(system_size[i] / ax_mag[i])
    if scale % 2 == 1:
        scale += 1
    s[i] = scale / 2
size = np.array( [[-s[0], s[0]], [-s[1], s[1]], [-s[2], s[2]]], dtype=np.int )

#Specify file name information
outfile = prototype.get('tag') + '-' 
for symbol in symbols:
    outfile += '-' + potential.elements(symbol)
outfile += '--' + disl_type

#Build dislocation-free system
base_file = outfile + '--base.dump' 
system_info = am.lammps.sys_gen(units =       potential.units(),
                                atom_style =  potential.atom_style(),
                                pbc =         (False, False, True),
                                ucell_box =   ucell.box(),
                                ucell_atoms = ucell.atoms(scale=True),
                                axes =        axes,
                                shift =       shift,
                                size =        size)

sys0 = am.lammps.create_sys(lammps_exe, system_info)
am.lammps.write_dump(base_file, sys0)  

print base_file + ' created'

fcc--Al--(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(0)(-1)]--base.dump created


###5.2 Stroh Calculations

This section uses the Stroh method to find the elasticity solution.  The pre-ln factor is printed and the elastic solution displacements and stresses are appended to the atoms in the dump file for the base system.

In [12]:
#Transform Burgers vector and elastic constant matrix
burgers = np.array([ucell.box('a') * dislocations.get(disl_type, 'burgers')[0],
                    ucell.box('b') * dislocations.get(disl_type, 'burgers')[1],
                    ucell.box('c') * dislocations.get(disl_type, 'burgers')[2]])

b = T.dot(burgers)
C = c_transform( c_mn_to_c_ijkl(cij), T)

#Run Stroh calculations
strohdata = stroh_setup(C)
pre_ln = stroh_preln(b, strohdata)
print 'The pre-ln energy factor is %f eV' % pre_ln

for i in xrange(sys0.natoms()):
    pos_x = sys0.atoms(i, 'pos', 0)
    pos_y = sys0.atoms(i, 'pos', 1)
    
    disp = stroh_disp_point(pos_x, pos_y, b, strohdata)
    stress = stroh_stress_point(pos_x, pos_y, b, C, strohdata)
    
    sys0.atoms(i, 'Stroh_displacement', disp)
    sys0.atoms(i, 'Stroh_stress', stress)

am.lammps.write_dump(base_file, sys0)

The pre-ln energy factor is 0.108118 eV


###5.3 Dislocation System Construction

This section creates a system containing a dislocation monopole by shifting atoms according to the Stroh displacement.

In [13]:
#Define run parameters
b_width = 3 * ucell.box('a')
units = potential.units()
atom_style = potential.atom_style()
sys1 = deepcopy(sys0)

for i in xrange(sys0.natoms()):
    newpos = sys0.atoms(i, 'pos') + sys0.atoms(i, 'Stroh_displacement')
    sys1.atoms(i, 'pos', newpos)

#Apply boundary conditions
boundaryfix(sys1, b_width, 'circle')
moving_atypes = []
for i in xrange(len(symbols)):
    moving_atypes.append(i+1)
symbols += symbols 
sys1.wrap()

#Relax system
pair_info = potential.pair_info(symbols)
system_info = am.lammps.write_data('disl.dat', sys1, units=units, atom_style=atom_style)

with open('disl_relax.in', 'w') as f:
    f.write(disl_relax_script(system_info, pair_info, moving_atypes, temp=temperature))

disl_file = outfile + '--disl.dump'
data = am.lammps.log_extract(subprocess.check_output(lammps_exe + ' -in disl_relax.in',shell=True))
sys2 = am.lammps.read_dump('atom.'+data[-1][0])
am.lammps.write_dump(disl_file, sys2)

print disl_file + ' created'

fcc--Al--(a-2)[(1)(0)(-1)]--((1)(1)(1))--u[(1)(0)(-1)]--disl.dump created
